In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 9
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
# Class 1
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
delay.set_service(jobclass1, Exp(0.5))

# Class 2
queue1.set_service(jobclass2, Exp(1.0))
queue2.set_service(jobclass2, Exp(2.0))
delay.set_service(jobclass2, Exp(0.2))

In [5]:
# Set routing matrix
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
 Delay1   class1 1.0603 1.0603  1.9953  1.9953 0.5190 0.5190
 Delay1   class2 2.2419 2.2419  5.0054  5.0054 0.4486 0.4486
 Queue1   class1 8.9397 0.5441 16.9465 16.9465 0.5190 0.5190
 Queue1   class2 7.7581 0.4576 16.9936 16.9936 0.4463 0.4463
 Queue2   class1 0.4987 0.2644  0.9126  0.9126 0.5219 0.5219
 Queue2   class2 0.4349 0.2239  0.9698  0.9698 0.4485 0.4485
   Join   class1 8.4208 0.9997  7.9349  7.9349 0.5190 0.5190
   Join   class2 7.3160 0.9990  8.1515  8.1515 0.4463 0.4463
SOLVER 2: MVA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.7813 1.7813 2.0000  2.0000 0.8907 0.8907
 Delay1   class2 4.2498 4.2498 5.0000  5.0000 0.8500 0.8500
 Queue1   class1 8.1466 0.8907 9.1466  9.1466 0.8907 0.8907
 Queue1   class2 5.6649 0.8500 6.6649  6.6649 0.8500 0.8500
 Queue2   class1 0.8029 0.4453 0.9014  0.9014 0.8907 0.8907
 Queue2   class2 0.7391 0.4250 0.8695  0.8695 0.8500 0.8500
   Join   class1 